## Install the geocoder

In [1]:
!pip install geocoder


## Get the survey data

- [survey](https://docs.google.com/forms/d/1oF7us4ASuFpF6iOhn4g5MnXf4Z8E5EPaIe2mMdPw_cM/edit)
- [data](https://docs.google.com/spreadsheets/d/1qPtvXZJ-IoNXdbVcPxr4buVLUONGsYS7-xNGqPFP144/edit#gid=696092516)


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSg8WVXVb47Tri6-JLpYZ-rxXUr3iX5HRD23rzK5XmABGpQI3jfSZk9btVTiguQZIgIWAmecSjbXvUM/pub?gid=696092516&single=true&output=csv')
                 

In [4]:
df

,Timestamp,Where do you live? Enter a zipcode.,"If it were up to you, how would you want to work moving forward?",Email Address
0,10/8/2021 10:30:48,90230,hybrid,NaN
1,10/8/2021 10:32:54,90095,100% in person,NaN
2,10/8/2021 10:33:11,santa monica,100% remote,NaN


## Rename columns

In [5]:
df.columns=['timestamp','zip','work','email']

In [6]:
df

,timestamp,zip,work,email
0,10/8/2021 10:30:48,90230,hybrid,NaN
1,10/8/2021 10:32:54,90095,100% in person,NaN
2,10/8/2021 10:33:11,santa monica,100% remote,NaN


## Geocode the zipcodes

In [7]:
import geocoder

In [8]:
df['lat'] = ''
df['lon'] = ''

In [9]:
for index, row in df.iterrows():
    g = geocoder.mapbox(row.zip,key='pk.eyJ1IjoiY2h3b2lwcm9qZWN0IiwiYSI6ImNrdTRjZXkwaTRwaWwycXBtY290MjA5d3gifQ.iyENw-MBBY3_9tMPv7frFg')
    print(g.latlng)
    df.loc[index,'lat'] = g.latlng[0]
    df.loc[index,'lon'] = g.latlng[1]
    

[33.98, -118.39]
[34.07, -118.44]
[34.0196, -118.487]


In [10]:
df

,timestamp,zip,work,email,lat,lon
0,10/8/2021 10:30:48,90230,hybrid,NaN,33.98,-118.39
1,10/8/2021 10:32:54,90095,100% in person,NaN,34.07,-118.44
2,10/8/2021 10:33:11,santa monica,100% remote,NaN,34.0196,-118.487


### Get average lat/lon's

In [11]:
# average latitude
latitude = df.lat.mean()
latitude

34.023199999999996

In [12]:
# average longitude
longitude = df.lon.mean()
longitude

-118.43900000000001

## Map it!

In [13]:
import folium

In [14]:
from folium.plugins import FloatImage

# image_file = 'image.PNG'

# FloatImage(image_file, bottom=0, left=86).add_to(mymap)

In [49]:
# map setup
m = folium.Map(location = [34.0703726,-118.4453799], tiles="Stamen Terrain", zoom_start=10)

# add radius circles around ucla
radii = [40,30,20,10]
for radius in radii:
    folium.Circle(
        popup=str(radius)+" km radius from UCLA",
        tooltip=str(radius)+" km radius from UCLA",
        radius=radius*1000,
        location=[34.0703726,-118.4453799],
        color="crimson",
        fill=True,
        opacity=0.3,
        fill_opacity=0.05
    ).add_to(m)

# add markers with tooltips colored by desired work mode
for index, row in df.iterrows():
    tooltip = 'I live in zipcode '+str(row.zip)+' and want to work '+row.work
    # color
    if row.work == '100% remote':
        color = 'red'
    elif row.work == 'hybrid':
        color = 'darkblue'
    else:
        color = 'green'
    folium.Marker([row.lat, row.lon], 
                  popup=row.work, 
                  tooltip=tooltip,
                  icon=folium.Icon(color=color,icon='user'),
                 ).add_to(m)
    
FloatImage('https://raw.githubusercontent.com/yohman/interactive-engagement/main/legend3.png',left=10).add_to(m)
# output the map
m

In [16]:
# save the interactive maps as an html file
m.save('index.html')

In [23]:
# Import the os module
import os

# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))

Current working directory: /home/jovyan/testroom/interactive-engagement
